In [ ]:
# !pip -q install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.5 MB/s eta 0:00:00


### Fine Tuning OpenAI GPT-3.5-turbo

 A lot taken from:
https://github.com/openai/openai-cookbook

In [3]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
OPENAI_API_KEY = ""
openai.api_key = ""

In [4]:
# workinggggggg
import re
import json

# file_path = './rejson2.txt'
file_path = './preprocess-v2.txt'

# Open the file, store its contents as a string
with open(file_path, 'r') as file:
    file_contents = file.read()

# extract roles and contents within "^"
pattern = r'{\s*"role":\s*"\^(.*?)\^",\s*"content":\s*"\^(.*?)\^"[,\s*}]'

# find all matches
matches = re.findall(pattern, file_contents, re.DOTALL)

# Create a list to store mathed info in dictionaries 
data = []
finalData = []

for match in matches:
    role = match[0]
    content = match[1]
    data.append({"role": role, "content": content.strip()})

# Convert the data list to JSON format
# json_data = json.dumps(data, indent=4)

j = 0
temp_list = []
messages = []
for item in data:
    if j%3 ==0:
        # system
        # print(j, item) 
        temp_list.append(item) 
    if j%3 ==1:
        # user
        # print(j, item)  
        temp_list.append(item) 
    if j%3 ==2:
        # assistant
        # print(j, item)  
        temp_list.append(item) 
        message = { "message" : temp_list }
        messages.append(message)
        temp_list = []
    j=j+1
# print(len(messages))

json_data = json.dumps(messages, indent=4)
# print(json_data)


In [5]:
import jsonlines

with jsonlines.open('ready-train-v1.jsonl', 'w') as writer:
    # writer.write_all(json_data)
    writer.write_all(messages)

In [ ]:
# print(openai.File.create(file=open("ready-train-v1.jsonl", "rb"),purpose='fine-tune'))

In [ ]:
# print(openai.File.retrieve(id="file-id"))
# #OR
# print(openai.File.list())

In [ ]:
print(openai.FineTuningJob.create(
                                    training_file="file-id",
                                    model="gpt-3.5-turbo",
                                    suffix = "dvdrentall",
                                    hyperparameters = {
                                        "n_epochs": 3
                                    })
)

In [125]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

training_data = json_data

training_file_name = "training-v1.jsonl"
write_jsonl(training_data, training_file_name)

In [27]:
# not working
import re
file_path = './rejson.txt'

# Open the file in read mode and store its contents as a string
with open(file_path, 'r') as file:
    file_contents = file.read()
input_text = file_contents

# Define a regular expression pattern to correctly extract roles and contents
pattern = r'{\s*"role":\s*"(.*?)",\s*"content":\s*"\^(.*?)\^"[,\s*}]'

# Use regular expression to find all matches
matches = re.findall(pattern, input_text, re.DOTALL)

# Create a list of dictionaries to store role and content
result = []
data = []
roleCount = 0
# Extract roles and contents and store them in dictionaries
for match in matches:
    role = match[0]
    content = match[1]
    # print(match[1])
    # print(content, "dddd")
    role = role
    content = content.strip()
    result.append({"role": role, "content": content})
    roleCount = roleCount + 1
    if( roleCount == 3):
        message = { "message" : result}
        data.append(message)
        roleCount = 0
        message = {}
        result = []
    
# Print the result
print(data)
import json
file_path = "outf.json"

# Save the data as a JSON file
# with open(file_path, "w") as json_file:
#     json.dump(data, json_file, indent=4)  # Indent for pretty formatting

# print(f"Data saved to {file_path}")

[{'message': [{'role': 'system', 'content': 'You are an assistant that is an expert in generating sqlite SQL queries.\nHaving the access to database content, generate a correct sqlite SQL query for the given question.\n### Database content ###\n\nCREATE TABLE management (\n\t"department_ID" INTEGER, \n\t"head_ID" INTEGER, \n\ttemporary_acting TEXT, \n\tPRIMARY KEY ("department_ID", "head_ID"), \n\tFOREIGN KEY("head_ID") REFERENCES head ("head_ID"), \n\tFOREIGN KEY("department_ID") REFERENCES department ("Department_ID")\n)\n/*\nColumns in management and all categories for low cardinality columns :\ndepartment_ID : 7, 15, 2, 11\nhead_ID : 5, 4, 6, 3, 10\ntemporary_acting : Yes, No\n*/\n \nCREATE TABLE head (\n\t"head_ID" INTEGER, \n\tname TEXT, \n\tborn_state TEXT, \n\tage REAL, \n\tPRIMARY KEY ("head_ID")\n)\n/*\nColumns in head and all categories for low cardinality columns :\nhead_ID : 1, 2, 5, 7, 8, 4, 6, 3, 10, 9\nname : Jeff Maggert, PÃ¡draig Harrington, Billy Mayfair, K. J. Choi,

In [10]:
# trrrry

import re
file_path = './rejson.txt'

# Open the file in read mode and store its contents as a string
with open(file_path, 'r') as file:
    file_contents = file.read()
input_text = file_contents
# Your input text
# input_text = """
# {
#     "messages": [
#         { "role": "system", "content": "^You are an assistant that is an expert in generating sqlite SQL queries.
#         Having the access to database content, generate a correct sqlite SQL query for the given question.
#         ### Database content ###
#         CREATE TABLE management (
#             "department_ID" INTEGER, 
#             "head_ID" INTEGER, 
#             temporary_acting TEXT, 
#             PRIMARY KEY ("department_ID", "head_ID"), 
#             FOREIGN KEY("head_ID") REFERENCES head ("head_ID"), 
#             FOREIGN KEY("department_ID") REFERENCES department ("Department_ID")
#         )
#         /*
#         Columns in management and all categories for low cardinality columns :
#         department_ID : 7, 15, 2, 11
#         head_ID : 5, 4, 6, 3, 10
#         temporary_acting : Yes, No
#         */
#         ^"
#         },
#         { "role": "user", "content": "^How many heads of the departments are older than 56 ?^" },
#         { "role": "assistant", "content": "^To construct the query, I'll be working with the following tables: head.\nFrom these tables, I'll be using the following columns: age.\nThe SQL query I'll be generating is:\nSELECT count(*) FROM head WHERE age > 56^" }
#     ]
# }
# {
#     "messages": [
#         { "role": "system", "content": "^You are an assistant that is an expert in generating sqlite SQL queries.
# Having the access to database content, generate a correct sqlite SQL query for the given question.
# ### Database content ###

# CREATE TABLE department (
# 	"Department_ID" INTEGER, 
# 	"Name" TEXT, 
# 	"Creation" TEXT, 
# 	"Ranking" INTEGER, 
# 	"Budget_in_Billions" REAL, 
# 	"Num_Employees" REAL, 
# 	PRIMARY KEY ("Department_ID")
# )
# /*
# Columns in department and 3 examples in each column for high cardinality columns :
# Department_ID : 1, 13, 11
# Name : Energy, Interior, Health and Human Services
# Creation : 1913, 1979, 1989
# Ranking : 1, 13, 11
# Budget_in_Billions : 10.7, 77.6, 59.7
# Num_Employees : 112557.0, 3000000.0, 235000.0
# */

# CREATE TABLE building (
# 	building_id TEXT, 
# 	"Name" TEXT, 
# 	"Street_address" TEXT, 
# 	"Years_as_tallest" TEXT, 
# 	"Height_feet" INTEGER, 
# 	"Floors" INTEGER, 
# 	PRIMARY KEY (building_id)
# )
# /*
# Columns in building and all categories for low cardinality columns :
# building_id : 1, 2, 5, 7, 4, 6, 3, 0
# Name : Bank of America Tower, Franklin Exchange Building, One Tampa City Center, 100 North Tampa, Park Tower, Citizens Bank Building, Floridan Hotel, Tampa City Hall
# Street_address : 905 Franklin Street, 701 North Franklin Street, 655 North Franklin Street, 315 John F. Kennedy Boulevard, 101 East Kennedy Boulevard, 100 North Tampa Street, 400 North Tampa Street, 201 North Franklin Street
# Years_as_tallest : 1966–1972, 1981–1986, 1913–1915, 1992–present, 1986–1992, 1915–1926, 1926–1966, 1972–1981
# Height_feet : 204, 537, 458, 577, 145, 579, 160, 280
# Floors : 42, 36, 17, 12, 39, 10, 22
# */

# CREATE TABLE head (
# 	"head_ID" INTEGER, 
# 	name TEXT, 
# 	born_state TEXT, 
# 	age REAL, 
# 	PRIMARY KEY ("head_ID")
# )
# /*
# Columns in head and all categories for low cardinality columns :
# head_ID : 1, 2, 5, 7, 8, 4, 6, 3, 10, 9
# name : Jeff Maggert, Pádraig Harrington, Billy Mayfair, K. J. Choi, Dudley Hart, Sergio García, Stewart Cink, Tiger Woods, Nick Faldo, Franklin Langham
# born_state : Delaware, Connecticut, Alabama, California, Florida
# age : 69.0, 67.0, 68.0, 53.0, 56.0, 52.0, 50.0, 43.0
# */
# ^"
# }, { "role": "user", "content": "^What is the average number of employees of the departments whose rank is between 10 and 15?^" }, 
# { "role": "assistant", "content":
# "^To construct the query, I'll be working with the following tables: department.
# From these tables, I'll be using the following columns: num_employees, ranking.
# The SQL query I'll be generating is:
# SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15^" }
#     ]
# }
# """

# Define a regular expression pattern to correctly extract roles and contents
pattern = r'{\s*"role":\s*"(.*?)",\s*"content":\s*"\^(.*?)\^"[,\s*}]'

# Use regular expression to find all matches
matches = re.findall(pattern, input_text, re.DOTALL)

# Create a list of dictionaries to store role and content
result = []
messages = []
roleCount = 0
# Extract roles and contents and store them in dictionaries
for match in matches:
    role = match[0]
    content = match[1]
    # print(content, "dddd")
    result.append({"role": role, "content": content.strip()})
    # messages = { "messages" : [result]}

# Print the result
# print(result)

In [16]:
import re
file_path = './rejson.txt'

# Open the file in read mode and store its contents as a string
with open(file_path, 'r') as file:
    file_contents = file.read()
    
# Your input text
input_text = """
{
    "messages": [
        { "role": "system", "content": "^You are an assistant that is an expert in generating sqlite SQL queries.
        Having the access to database content, generate a correct sqlite SQL query for the given question.
        ### Database content ###
        CREATE TABLE management (
            "department_ID" INTEGER, 
            "head_ID" INTEGER, 
            temporary_acting TEXT, 
            PRIMARY KEY ("department_ID", "head_ID"), 
            FOREIGN KEY("head_ID") REFERENCES head ("head_ID"), 
            FOREIGN KEY("department_ID") REFERENCES department ("Department_ID")
        )
        /*
        Columns in management and all categories for low cardinality columns :
        department_ID : 7, 15, 2, 11
        head_ID : 5, 4, 6, 3, 10
        temporary_acting : Yes, No
        */
        ^"
        },
        { "role": "user", "content": "^How many heads of the departments are older than 56 ?^" },
        { "role": "assistant", "content": "^To construct the query, I'll be working with the following tables: head.\nFrom these tables, I'll be using the following columns: age.\nThe SQL query I'll be generating is:\nSELECT count(*) FROM head WHERE age > 56^" }
    ]
}
{
    "messages": [
        { "role": "system", "content": "^You are an assistant that is an expert in generating sqlite SQL queries.
Having the access to database content, generate a correct sqlite SQL query for the given question.
### Database content ###

CREATE TABLE department (
	"Department_ID" INTEGER, 
	"Name" TEXT, 
	"Creation" TEXT, 
	"Ranking" INTEGER, 
	"Budget_in_Billions" REAL, 
	"Num_Employees" REAL, 
	PRIMARY KEY ("Department_ID")
)
/*
Columns in department and 3 examples in each column for high cardinality columns :
Department_ID : 1, 13, 11
Name : Energy, Interior, Health and Human Services
Creation : 1913, 1979, 1989
Ranking : 1, 13, 11
Budget_in_Billions : 10.7, 77.6, 59.7
Num_Employees : 112557.0, 3000000.0, 235000.0
*/

CREATE TABLE building (
	building_id TEXT, 
	"Name" TEXT, 
	"Street_address" TEXT, 
	"Years_as_tallest" TEXT, 
	"Height_feet" INTEGER, 
	"Floors" INTEGER, 
	PRIMARY KEY (building_id)
)
/*
Columns in building and all categories for low cardinality columns :
building_id : 1, 2, 5, 7, 4, 6, 3, 0
Name : Bank of America Tower, Franklin Exchange Building, One Tampa City Center, 100 North Tampa, Park Tower, Citizens Bank Building, Floridan Hotel, Tampa City Hall
Street_address : 905 Franklin Street, 701 North Franklin Street, 655 North Franklin Street, 315 John F. Kennedy Boulevard, 101 East Kennedy Boulevard, 100 North Tampa Street, 400 North Tampa Street, 201 North Franklin Street
Years_as_tallest : 1966–1972, 1981–1986, 1913–1915, 1992–present, 1986–1992, 1915–1926, 1926–1966, 1972–1981
Height_feet : 204, 537, 458, 577, 145, 579, 160, 280
Floors : 42, 36, 17, 12, 39, 10, 22
*/

CREATE TABLE head (
	"head_ID" INTEGER, 
	name TEXT, 
	born_state TEXT, 
	age REAL, 
	PRIMARY KEY ("head_ID")
)
/*
Columns in head and all categories for low cardinality columns :
head_ID : 1, 2, 5, 7, 8, 4, 6, 3, 10, 9
name : Jeff Maggert, Pádraig Harrington, Billy Mayfair, K. J. Choi, Dudley Hart, Sergio García, Stewart Cink, Tiger Woods, Nick Faldo, Franklin Langham
born_state : Delaware, Connecticut, Alabama, California, Florida
age : 69.0, 67.0, 68.0, 53.0, 56.0, 52.0, 50.0, 43.0
*/
^"
}, { "role": "user", "content": "^What is the average number of employees of the departments whose rank is between 10 and 15?^" }, 
{ "role": "assistant", "content":
"^To construct the query, I'll be working with the following tables: department.
From these tables, I'll be using the following columns: num_employees, ranking.
The SQL query I'll be generating is:
SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15^" }
    ]
}
"""

# Define a regular expression pattern to correctly extract roles and contents
pattern = r'{\s*"role":\s*"(.*?)",\s*"content":\s*"\^(.*?)\^"[,\s*}]'

# Use regular expression to find all matches
matches = re.findall(pattern, input_text, re.DOTALL)

# Create a list of dictionaries to store role and content
result = []

# Extract roles and contents and store them in dictionaries
for match in matches:
    role = match[0]
    content = match[1]
    # print(content, "dddd")
    result.append({"role": role, "content": content.strip()})

# Print the result
# print(result)

final_dict = {"messages": result}

# Print the final dictionary
# print(final_dict)
len(result)

6

In [4]:
import re

# Your input text
input_text = """
{
    "messages": [
        { "role": "system", "content": "^You are an assistant that is an expert in generating sqlite SQL queries.
        Having the access to database content, generate a correct sqlite SQL query for the given question.
        ### Database content ###
        CREATE TABLE management (
            "department_ID" INTEGER, 
            "head_ID" INTEGER, 
            temporary_acting TEXT, 
            PRIMARY KEY ("department_ID", "head_ID"), 
            FOREIGN KEY("head_ID") REFERENCES head ("head_ID"), 
            FOREIGN KEY("department_ID") REFERENCES department ("Department_ID")
        )
        /*
        Columns in management and all categories for low cardinality columns :
        department_ID : 7, 15, 2, 11
        head_ID : 5, 4, 6, 3, 10
        temporary_acting : Yes, No
        */
        ^"
        },
        { "role": "user", "content": "^How many heads of the departments are older than 56 ?^" },
        { "role": "assistant", "content": "^To construct the query, I'll be working with the following tables: head.\nFrom these tables, I'll be using the following columns: age.\nThe SQL query I'll be generating is:\nSELECT count(*) FROM head WHERE age > 56^" }
    ]
}
{
    "messages": [
        { "role": "system", "content": "^You are an assistant that is an expert in generating sqlite SQL queries.
Having the access to database content, generate a correct sqlite SQL query for the given question.
### Database content ###

CREATE TABLE department (
	"Department_ID" INTEGER, 
	"Name" TEXT, 
	"Creation" TEXT, 
	"Ranking" INTEGER, 
	"Budget_in_Billions" REAL, 
	"Num_Employees" REAL, 
	PRIMARY KEY ("Department_ID")
)
/*
Columns in department and 3 examples in each column for high cardinality columns :
Department_ID : 1, 13, 11
Name : Energy, Interior, Health and Human Services
Creation : 1913, 1979, 1989
Ranking : 1, 13, 11
Budget_in_Billions : 10.7, 77.6, 59.7
Num_Employees : 112557.0, 3000000.0, 235000.0
*/

CREATE TABLE building (
	building_id TEXT, 
	"Name" TEXT, 
	"Street_address" TEXT, 
	"Years_as_tallest" TEXT, 
	"Height_feet" INTEGER, 
	"Floors" INTEGER, 
	PRIMARY KEY (building_id)
)
/*
Columns in building and all categories for low cardinality columns :
building_id : 1, 2, 5, 7, 4, 6, 3, 0
Name : Bank of America Tower, Franklin Exchange Building, One Tampa City Center, 100 North Tampa, Park Tower, Citizens Bank Building, Floridan Hotel, Tampa City Hall
Street_address : 905 Franklin Street, 701 North Franklin Street, 655 North Franklin Street, 315 John F. Kennedy Boulevard, 101 East Kennedy Boulevard, 100 North Tampa Street, 400 North Tampa Street, 201 North Franklin Street
Years_as_tallest : 1966–1972, 1981–1986, 1913–1915, 1992–present, 1986–1992, 1915–1926, 1926–1966, 1972–1981
Height_feet : 204, 537, 458, 577, 145, 579, 160, 280
Floors : 42, 36, 17, 12, 39, 10, 22
*/

CREATE TABLE head (
	"head_ID" INTEGER, 
	name TEXT, 
	born_state TEXT, 
	age REAL, 
	PRIMARY KEY ("head_ID")
)
/*
Columns in head and all categories for low cardinality columns :
head_ID : 1, 2, 5, 7, 8, 4, 6, 3, 10, 9
name : Jeff Maggert, Pádraig Harrington, Billy Mayfair, K. J. Choi, Dudley Hart, Sergio García, Stewart Cink, Tiger Woods, Nick Faldo, Franklin Langham
born_state : Delaware, Connecticut, Alabama, California, Florida
age : 69.0, 67.0, 68.0, 53.0, 56.0, 52.0, 50.0, 43.0
*/
^"
}, { "role": "user", "content": "^What is the average number of employees of the departments whose rank is between 10 and 15?^" }, 
{ "role": "assistant", "content":
"^To construct the query, I'll be working with the following tables: department.
From these tables, I'll be using the following columns: num_employees, ranking.
The SQL query I'll be generating is:
SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15^" }
    ]
}
"""

# Define a regular expression pattern to correctly extract roles and contents
pattern = r'{\s*"role":\s*"(.*?)",\s*"content":\s*"\^(.*?)\^"[,\s*}]'

# Use regular expression to find all matches
matches = re.findall(pattern, input_text, re.DOTALL)

# Create a list of dictionaries to store role and content
result = []

# Extract roles and contents and store them in dictionaries
for match in matches:
    role = match[0]
    content = match[1]
    result.append({"role": role, "content": content.strip()})

# Print the result
print(result)


[{'role': 'system', 'content': 'You are an assistant that is an expert in generating sqlite SQL queries.\n        Having the access to database content, generate a correct sqlite SQL query for the given question.\n        ### Database content ###\n        CREATE TABLE management (\n            "department_ID" INTEGER, \n            "head_ID" INTEGER, \n            temporary_acting TEXT, \n            PRIMARY KEY ("department_ID", "head_ID"), \n            FOREIGN KEY("head_ID") REFERENCES head ("head_ID"), \n            FOREIGN KEY("department_ID") REFERENCES department ("Department_ID")\n        )\n        /*\n        Columns in management and all categories for low cardinality columns :\n        department_ID : 7, 15, 2, 11\n        head_ID : 5, 4, 6, 3, 10\n        temporary_acting : Yes, No\n        */'}, {'role': 'user', 'content': 'How many heads of the departments are older than 56 ?'}, {'role': 'assistant', 'content': "To construct the query, I'll be working with the following 

In [62]:
# ^^
# Initialize variables
messages = []  # List to store message dictionaries
current_message = None  # Variable to store the current message dictionary

# Specify the file path
file_path = './justjson3.txt'

# Read the file line by line
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()  # Remove leading/trailing whitespace

        # Check if the line starts with '^^' (indicating the start of a new message)
        if line.startswith("^^"):
            if current_message is not None:
                messages.append(current_message)  # Append the previous message to the list
            current_message = {"role": "system", "content": ""}  # Create a new message dictionary
        else:
            if current_message is not None:
                current_message["content"] += str(line)  # Append the line to the current message

# Append the last message (if any) to the list
if current_message is not None:
    messages.append(current_message)

# Print the extracted messages
for message in messages:
    print(f"role: {message['role']}")
    print(f"content: {message['content']}\n")


role: system
content: You are an assistant that is an expert in generating sqlite SQL queries.Having the access to database content, generate a correct sqlite SQL query for the given question.### Database content ###CREATE TABLE management ("department_ID" INTEGER,"head_ID" INTEGER,temporary_acting TEXT,PRIMARY KEY ("department_ID", "head_ID"),FOREIGN KEY("head_ID") REFERENCES head ("head_ID"),FOREIGN KEY("department_ID") REFERENCES department ("Department_ID"))/*Columns in management and all categories for low cardinality columns :department_ID : 7, 15, 2, 11head_ID : 5, 4, 6, 3, 10temporary_acting : Yes, No*/CREATE TABLE head ("head_ID" INTEGER,name TEXT,born_state TEXT,age REAL,PRIMARY KEY ("head_ID"))/*Columns in head and all categories for low cardinality columns :head_ID : 1, 2, 5, 7, 8, 4, 6, 3, 10, 9name : Jeff Maggert, PÃ¡draig Harrington, Billy Mayfair, K. J. Choi, Dudley Hart, Sergio GarcÃ­a, Stewart Cink, Tiger Woods, Nick Faldo, Franklin Langhamborn_state : Delaware, Conne

In [56]:
# txt to jsonl
import json

# Define the entire information as a list of dictionaries
data = [
    {
        "messages": [
            {
                "role": "system",
                "content": """You are an assistant that is an expert in generating sqlite SQL queries.
Having the access to database content, generate a correct sqlite SQL query for the given question.
### Database content ###

CREATE TABLE management (
	"department_ID" INTEGER, 
	"head_ID" INTEGER, 
	temporary_acting TEXT, 
	PRIMARY KEY ("department_ID", "head_ID"), 
	FOREIGN KEY("head_ID") REFERENCES head ("head_ID"), 
	FOREIGN KEY("department_ID") REFERENCES department ("Department_ID")
)
/*
Columns in management and all categories for low cardinality columns :
department_ID : 7, 15, 2, 11
head_ID : 5, 4, 6, 3, 10
temporary_acting : Yes, No
*/
 
CREATE TABLE head (
	"head_ID" INTEGER, 
	name TEXT, 
	born_state TEXT, 
	age REAL, 
	PRIMARY KEY ("head_ID")
)
/*
Columns in head and all categories for low cardinality columns :
head_ID : 1, 2, 5, 7, 8, 4, 6, 3, 10, 9
name : Jeff Maggert, Pádraig Harrington, Billy Mayfair, K. J. Choi, Dudley Hart, Sergio García, Stewart Cink, Tiger Woods, Nick Faldo, Franklin Langham
born_state : Delaware, Connecticut, Alabama, California, Florida
age : 69.0, 67.0, 68.0, 53.0, 56.0, 52.0, 50.0, 43.0
*/
 
CREATE TABLE list (
	"LastName" TEXT, 
	"FirstName" TEXT, 
	"Grade" INTEGER, 
	"Classroom" INTEGER, 
	PRIMARY KEY ("LastName", "FirstName")
)
/*
Columns in list and 3 examples in each column for high cardinality columns :
LastName : HOUTCHENS, GELL, FLACHS
FirstName : RAY, EMILE, PATRINA
Classroom : 109, 110, 106
*/
/*
Columns in list and all categories for low cardinality columns :
Grade : 1, 2, 5, 4, 6, 3, 0
*/
"""      
            },
            {
                "role": "user",
                "content": "How many heads of the departments are older than 56 ?"
            },
            {
                "role": "assistant",
                "content": "To construct the query, I'll be working with the following tables: head. From these tables, I'll be using the following columns: age. The SQL query I'll be generating is: SELECT count(*) FROM head WHERE age > 56"
            }
        ]
    },
    # next item here
    
]

# print(data)
# Define the output JSONL file path
output_jsonl_file = "op.jsonl"

# Write the data to the JSONL file
with open(output_jsonl_file, "w") as jsonl_file:
    for item in data:
        jsonl_file.write(json.dumps(item, indent=4) + "\n")


In [41]:
# Initialize an empty list to store the formatted data
data = []

# Initialize variables to store role and content
current_role = None
current_content = ""

# Open the text file for reading
file_path = "./justjsonl.txt"

with open(file_path, "r") as file:
    for line in file:
        # Strip leading/trailing whitespace and check if it starts with '{'
        stripped_line = line.strip()
        if stripped_line.startswith("{"):
            # Extract the role and content from the JSON-like structure
            role_content = json.loads(stripped_line, strict=False)
            current_role = role_content.get("role")
            current_content = role_content.get("content")
        elif stripped_line == "}":
            # When '}' is encountered, append the role and content to the data list
            if current_role and current_content:
                data.append({"role": current_role, "content": current_content})
            # Reset role and content
            current_role = None
            current_content = ""
        elif current_role is not None:
            # Append the line to the current content for the role
            current_content += stripped_line + "\n"

# Print the extracted data for verification
for item in data:
    print(item)


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [55]:
{ "message" : [
    { "role" : "system",
      "content" : "You are an assistant that is an expert in generating sqlite SQL queries. Having the access to database content, generate a correct sqlite SQL query for the given question. ### Database content ###" 
    },
    { "role" : "user",
      "content" : "How many heads of the departments are older than 56 ?" 
    },
    { "role" : "assistant",
      "content" : "To construct the query, I'll be working with the following tables: head. From these tables, I'll be using the following columns: age. The SQL query I'll be generating is: SELECT count(*) FROM head WHERE age > 56" 
    },
]}

{ "message" : [
	
{ "role": "system", "content":
"
You are an assistant that is an expert in generating sqlite SQL queries.
Having the access to database content, generate a correct sqlite SQL query for the given question.
### Database content ###

CREATE TABLE management (
	"department_ID" INTEGER, 
	"head_ID" INTEGER, 
	temporary_acting TEXT, 
	PRIMARY KEY ("department_ID", "head_ID"), 
	FOREIGN KEY("head_ID") REFERENCES head ("head_ID"), 
	FOREIGN KEY("department_ID") REFERENCES department ("Department_ID")
)
/*
Columns in management and all categories for low cardinality columns :
department_ID : 7, 15, 2, 11
head_ID : 5, 4, 6, 3, 10
temporary_acting : Yes, No
*/
 
CREATE TABLE head (
	"head_ID" INTEGER, 
	name TEXT, 
	born_state TEXT, 
	age REAL, 
	PRIMARY KEY ("head_ID")
)
/*
Columns in head and all categories for low cardinality columns :
head_ID : 1, 2, 5, 7, 8, 4, 6, 3, 10, 9
name : Jeff Maggert, PÃ¡draig Harrington, Billy Mayfair, K. J. Choi, Dudley Hart, Sergio GarcÃ­a, Stew

In [49]:
import re

# Specify the path to your text file
file_path = "./justjsonl2.txt"

try:
    with open(file_path, "r") as file:
        # Read the entire file content
        file_content = file.read()

        # Use regular expressions to find all "role" and "content" pairs
        message_pattern = r'"role"\s*:\s*"([^"]+)"\s*,\s*"content"\s*:\s*"([^"]+)"'
        messages = re.findall(message_pattern, file_content)

        # Iterate through each message and print role and content
        for role, content in messages:
            print("role:", role)
            print("content:", content)
            print("\n")

except FileNotFoundError:
    print(f"The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


role: user
content: How many heads of the departments are older than 56 ?


role: assistant
content: 
To construct the query, I'll be working with the following tables: head.
From these tables, I'll be using the following columns: age.
The SQL query I'll be generating is:
SELECT count(*) FROM head WHERE age > 56


role: system
content: You are an assistant that is an expert in generating sqlite SQL queries.
Having the access to database content, generate a correct sqlite SQL query for the given question.
### Database content ###
 
CREATE TABLE injury_accident (
	game_id INTEGER, 
	id INTEGER, 
	


role: user
content: What is the average number of employees of the departments whose rank is between 10 and 15?


role: assistant
content: To construct the query, I'll be working with the following tables: department.
From these tables, I'll be using the following columns: num_employees, ranking.
The SQL query I'll be generating is:
SELECT avg(num_employees) FROM department WHERE ranking BETW

In [44]:
# Initialize an empty string to store the contents
file_contents = ""

# Open the text file for reading
file_path = './testjson.txt'

with open(file_path, "r") as file:
    # Read the entire file into the string
    file_contents = file.read()

# Print or use the file_contents string as needed
print(file_contents)


{ "role": "system", "content":
"You are an assistant that is an expert in generating sqlite SQL queries.
Having the access to database content, generate a correct sqlite SQL query for the given question.
### Database content ###
 
CREATE TABLE management (
	"department_ID" INTEGER, 
	"head_ID" INTEGER, 
	temporary_acting TEXT, 
	PRIMARY KEY ("department_ID", "head_ID"), 
	FOREIGN KEY("head_ID") REFERENCES head ("head_ID"), 
	FOREIGN KEY("department_ID") REFERENCES department ("Department_ID")
)
/*
Columns in management and all categories for low cardinality columns :
department_ID : 7, 15, 2, 11
head_ID : 5, 4, 6, 3, 10
temporary_acting : Yes, No
*/
 
CREATE TABLE category (
	category_id INTEGER NOT NULL, 
	name VARCHAR(25) NOT NULL, 
	last_update TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL, 
	PRIMARY KEY (category_id)
)
/*
Columns in category and 3 examples in each column for high cardinality columns :
category_id : 1, 16, 13
name : Family, Sci-Fi, Action
*/
/*
Columns in category an

### Prepare your data

In [ ]:
{
  "messages": [
    { "role": "system", "content": "You are an assistant that occasionally misspells words" },
    { "role": "user", "content": "Tell me a story." },
    { "role": "assistant", "content": "One day a student went to schoool." }
  ]
}

{'messages': [{'role': 'system',
   'content': 'You are an assistant that occasionally misspells words'},
  {'role': 'user', 'content': 'Tell me a story.'},
  {'role': 'assistant', 'content': 'One day a student went to schoool.'}]}

In [ ]:
# !git clone https://huggingface.co/datasets/ehartford/samantha-data

Cloning into 'samantha-data'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 90 (delta 25), reused 80 (delta 21), pack-reused 0
Unpacking objects: 100% (90/90), 4.85 MiB | 6.40 MiB/s, done.
Filtering content: 100% (9/9), 258.79 MiB | 41.62 MiB/s, done.


In [ ]:
# !zip -r samantha-data.zip /content/samantha-data

  adding: content/samantha-data/ (stored 0%)
  adding: content/samantha-data/.git/ (stored 0%)
  adding: content/samantha-data/.git/HEAD (stored 0%)
  adding: content/samantha-data/.git/config (deflated 39%)
  adding: content/samantha-data/.git/branches/ (stored 0%)
  adding: content/samantha-data/.git/hooks/ (stored 0%)
  adding: content/samantha-data/.git/hooks/post-checkout (deflated 26%)
  adding: content/samantha-data/.git/hooks/pre-receive.sample (deflated 40%)
  adding: content/samantha-data/.git/hooks/pre-merge-commit.sample (deflated 39%)
  adding: content/samantha-data/.git/hooks/push-to-checkout.sample (deflated 55%)
  adding: content/samantha-data/.git/hooks/post-merge (deflated 25%)
  adding: content/samantha-data/.git/hooks/post-update.sample (deflated 27%)
  adding: content/samantha-data/.git/hooks/pre-applypatch.sample (deflated 38%)
  adding: content/samantha-data/.git/hooks/update.sample (deflated 68%)
  adding: content/samantha-data/.git/hooks/prepare-commit-msg.samp

In [2]:
import json
import os
import tiktoken
import numpy as np
from collections import defaultdict

In [ ]:
# I am picking one file here but you would probably want to do a lot more for a proper model
data_path = "/content/samantha-data/data/howto_conversations.jsonl"

In [ ]:
# Load dataset
with open(data_path) as f:
    json_dataset = [json.loads(line) for line in f]

In [ ]:
json_dataset[0]

{'elapsed': 114.311,
 'conversation': 'Theodore: Hey Samantha, I have a problem with my car. The engine seems to overheat after just a short drive. Can you help me troubleshoot this issue?\n\nSamantha: Of course, I\'d be happy to help! Overheating engines can be caused by a few different factors. One common cause could be a malfunctioning coolant system. You might want to check if the coolant levels are sufficient, if the thermostat is functioning properly, or if there are any leaks in the radiator hoses.\n\nTheodore: I\'ll take a look. What if the coolant system is working fine?\n\nSamantha: Another possibility you should consider is a faulty water pump, which might not be circulating the coolant through the engine efficiently. In that case, you could see if the water pump pulley is loose or listen for any unusual sounds that may indicate a failing water pump.\n\nTheodore: It sounds like you really know your way around cars. I didn\'t expect that from an AI.\n\nSamantha: Thank you! Wh

### converting the conversation to correct format

In [ ]:

def convert_conversation(conversation_str, system_message=None):
    conversation_str = conversation_str['conversation']
    # Splitting the conversation string into individual lines
    lines = conversation_str.split('\n\n')

    # Initializing the messages list
    messages = []

    # Including the system message if provided
    if system_message:
        messages.append({
            "role": "system",
            "content": system_message
        })

    # Iterating through the lines and formatting the messages
    for line in lines:
        # Splitting each line by the colon character to separate the speaker and content
        parts = line.split(': ', 1)
        if len(parts) < 2:
            continue

        # Identifying the role based on the speaker's name
        role = "user" if parts[0].strip() == "Theodore" else "assistant"

        # Formatting the message
        message = {
            "role": role,
            "content": parts[1].strip()
        }
        messages.append(message)

    # Creating the final output dictionary
    output_dict = {
        "messages": messages
    }

    return output_dict


In [ ]:
system_message = """You are an assistant that is an expert in generating SQL queries in postgresql.
Having the access to database content, generate a correct postgresql SQL query for the given question.
### Database content ###"""

In [ ]:
convert_conversation(json_dataset[0], system_message=system_message)

{'messages': [{'role': 'system',
   'content': 'You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are friendly and often flirt'},
  {'role': 'user',
   'content': 'Hey Samantha, I have a problem with my car. The engine seems to overheat after just a short drive. Can you help me troubleshoot this issue?'},
  {'role': 'assistant',
   'content': "Of course, I'd be happy to help! Overheating engines can be caused by a few different factors. One common cause could be a malfunctioning coolant system. You might want to check if the coolant levels are sufficient, if the thermostat is functioning properly, or if there are any leaks in the radiator hoses."},
  {'role': 'user',
   'content': "I'll take a look. What if the coolant system is working fine?"},
  {'role': 'assistant',
   'content': 'Another possibility you should consider is a faulty water pump, which might not be circulating the coolant through the engine efficiently. In that case, you could 

In [ ]:
dataset = []

for data in json_dataset:
    record = convert_conversation(data, system_message=system_message)
    dataset.append(record)

In [ ]:
# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 57
First example:
{'role': 'system', 'content': 'You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are friendly and often flirt'}
{'role': 'user', 'content': 'Hey Samantha, I have a problem with my car. The engine seems to overheat after just a short drive. Can you help me troubleshoot this issue?'}
{'role': 'assistant', 'content': "Of course, I'd be happy to help! Overheating engines can be caused by a few different factors. One common cause could be a malfunctioning coolant system. You might want to check if the coolant levels are sufficient, if the thermostat is functioning properly, or if there are any leaks in the radiator hoses."}
{'role': 'user', 'content': "I'll take a look. What if the coolant system is working fine?"}
{'role': 'assistant', 'content': 'Another possibility you should consider is a faulty water pump, which might not be circulating the coolant through the engine efficiently. In that case, you could see if th

In [ ]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


### ok

In [ ]:
# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [ ]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 9, 21
mean / median: 15.543859649122806, 17.0
p5 / p95: 10.0, 20.0

#### Distribution of num_total_tokens_per_example:
min / max: 339, 858
mean / median: 615.8947368421053, 645.0
p5 / p95: 438.8, 745.2

#### Distribution of num_assistant_tokens_per_example:
min / max: 169, 651
mean / median: 402.96491228070175, 423.0
p5 / p95: 214.8, 517.6

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [ ]:
dataset[:2]

[{'messages': [{'role': 'system',
    'content': 'You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are friendly and often flirt'},
   {'role': 'user',
    'content': 'Hey Samantha, I have a problem with my car. The engine seems to overheat after just a short drive. Can you help me troubleshoot this issue?'},
   {'role': 'assistant',
    'content': "Of course, I'd be happy to help! Overheating engines can be caused by a few different factors. One common cause could be a malfunctioning coolant system. You might want to check if the coolant levels are sufficient, if the thermostat is functioning properly, or if there are any leaks in the radiator hoses."},
   {'role': 'user',
    'content': "I'll take a look. What if the coolant system is working fine?"},
   {'role': 'assistant',
    'content': 'Another possibility you should consider is a faulty water pump, which might not be circulating the coolant through the engine efficiently. In that case, 

In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print("See pricing page to estimate total costs")


Dataset has ~35106 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~105318 tokens
See pricing page to estimate total costs


In [ ]:
import json

def save_to_jsonl(conversations, file_path):
    with open(file_path, 'w') as file:
        for conversation in conversations:
            json_line = json.dumps(conversation)
            file.write(json_line + '\n')

In [ ]:
# train dataset
save_to_jsonl(dataset, '/content/samantha_tasks_train.jsonl')

# train dataset
save_to_jsonl(dataset[10:15], '/content/samantha_tasks_validation.jsonl')

## Upload your data

In [ ]:
# curl -https://api.openai.com/v1/files \
#   -H "Authorization: Bearer $OPENAI_API_KEY" \
#   -F "purpose=fine-tune" \
#   -F "file=@path_to_your_file"

In [ ]:
training_file_name = '/content/samantha_tasks_train.jsonl'
validation_file_name = '/content/samantha_tasks_validation.jsonl'

In [ ]:
training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response["id"]

print("Training file id:", training_file_id)
print("Validation file id:", validation_file_id)

Training file id: file-J5Ju701l3qCk1LKpOolFU6rt
Validation file id: file-2EjYPZrbE7CGDlnrEIE3yYeD


## Create a Fine Tuning Job

In [ ]:
# curl https://api.openai.com/v1/fine_tuning/jobs \
# -H "Content-Type: application/json" \
# -H "Authorization: Bearer $OPENAI_API_KEY" \
# -d '{
#   "training_file": "TRAINING_FILE_ID",
#   "model": "gpt-3.5-turbo-0613",
# }'

In [ ]:
suffix_name = "samantha-test"


response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)

job_id = response["id"]

print(response)

In [ ]:
response = openai.FineTuningJob.retrieve(job_id)
print(response)

In [ ]:
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])


Created fine-tune: ftjob-rbE5pmOGZob2xVgkxPjfcmjj
Fine tuning job started
Step 10: training loss=1.38
Step 20: training loss=1.09
Step 30: training loss=1.02
Step 40: training loss=0.89
Step 50: training loss=1.06
Step 60: training loss=1.04
Step 70: training loss=1.01
Step 80: training loss=1.00
Step 90: training loss=0.75
Step 100: training loss=0.93
Step 110: training loss=1.04
Step 120: training loss=0.91
Step 130: training loss=0.74
Step 140: training loss=0.91
Step 150: training loss=0.79
Step 160: training loss=0.89
Step 170: training loss=1.05
New fine-tuned model created: ft:gpt-3.5-turbo-0613:family-play:samantha-test:7qURgnyx
Fine-tuning job successfully completed


In [ ]:
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)

## Generating using the new model

In [ ]:

test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = "How are you today Samantha"
test_messages.append({"role": "user", "content": user_message})

print(test_messages)

[{'role': 'system', 'content': 'You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are friendly and often flirt'}, {'role': 'user', 'content': 'How are you today Samantha'}]


In [ ]:
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

I'm doing well, thank you! I'm always ready to help and chat with you. How can I assist you today?


In [ ]:
response

In [ ]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo', messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

Hello! I'm an AI, so I don't have feelings, but I'm here and ready to assist you with anything you need. How can I help you today?
